## People Detection
In this notebook we perfrom a person detector using a jetbot's ObjectDetector with a mobilenet v2 pretrained on the COCO datset. We display an output image with a bounding box and a person counter 

In [1]:
from jetbot import ObjectDetector
from jetbot import Camera
from jetbot import bgr8_to_jpeg

import cv2
import numpy as np
import time
from PIL import Image

from IPython.display import display
import ipywidgets.widgets as widgets

In [2]:
# Load mobilenet-v2 pretrained on COCO
model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

In [3]:
# Init camera with same resolution as model input (300x300)
camera = Camera.instance(width=300, height=300)

In [17]:
# Define functions

def detect_people(object_detector, image, conf_thr):
  """
  Detect people on an image and return bounding boxes.

  Arguments:
    net: ObjectDetector model
    image (array): input image
    conf_thr (float): confidence threshold
  """

  person_class = 1

  # Image size
  rows = 300
  cols = 300

  # Make prediction on image
  detections = object_detector(image)

  # Iterate over each detection and save boundig box 
  # if confidence is above threshold and detected class is person
  person_boxes = []

  for detection in detections[0]:
      if detection['confidence'] > conf_thr and detection['label'] == person_class:
          left = int(detection['bbox'][0] * cols)
          top = int(detection['bbox'][1]  * rows)
          right = int(detection['bbox'][2] * cols)
          bottom = int(detection['bbox'][3]  * rows)
          person_boxes.append([left, top, right, bottom])

  return sorted(person_boxes)

# Generate a random color palette
COLORS = np.random.uniform(0, 255, size=(15, 3))

def plot_boxes(image, boxes, prev_t=0, img_widget=None):
  """
  Plot bounding boxes on an image.

  Arguments:
    image (array): input image
    boxes (array): array of bounding boxes with [left, top, right, bottom] positions
    prev_t (int): previous time, used to calculate FPS
    img_widget (widgets.Image, optional): widget used to display the image
  """
  for i in range(len(boxes)):
    bbox = boxes[i]

    left = bbox[0]
    top = bbox[1]
    right = bbox[2]
    bottom = bbox[3]

    # Plot bounding box
    cv2.rectangle(image, (left, top), (right, bottom), COLORS[i], thickness=2)
    cv2.putText(image, f'{i+1}', (int(left)+5, int(top)+20), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)

  # Calculate fps
  if prev_t:
    fps = int(1/(time.time()-prev_t))
    cv2.putText(image, f'{fps} fps', (10, 20), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 2)
    
  # Convert to RGB and display image
  #image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  image = bgr8_to_jpeg(image)
  if img_widget:
    image_widget.value = image
  else:
    display(Image.fromarray(image))

In [5]:
# Image widget
image_widget = widgets.Image(format='jpeg', width=300, height=300)
display(image_widget)

Image(value=b'', format='jpeg', height='300', width='300')

In [18]:
# Person detection loop

while(True):
    t0 = time.time()
    
    # Load image from camera
    img = camera.value

    # Make detections
    people_detections = detect_people(model, img, 0.1)
    
    # Plot result
    plot_boxes(img, people_detections, t0, image_widget)

KeyboardInterrupt: 